Data Privacy Final Project 

In [2]:
# Load the data and libraries
import pandas as pd
import numpy as np
import random
from scipy import stats
import matplotlib.pyplot as plt

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def laplace_mech_vec(vec, sensitivity, epsilon):
    return [v + np.random.laplace(loc=0, scale=sensitivity / epsilon) for v in vec]

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

#adult = pd.read_csv('https://github.com/ninabraddock/data-privacy-final-project/student_math_clean.csv')
df = pd.read_csv("student_math_clean.csv")
df.head(5).T

,0,1,2,3,4
student_id,1,2,3,4,5
school,GP,GP,GP,GP,GP
sex,F,F,F,F,F
age,18,17,15,15,16
address_type,Urban,Urban,Urban,Urban,Urban
family_size,Greater than 3,Greater than 3,Less than or equal to 3,Greater than 3,Greater than 3
parent_status,Apart,Living together,Living together,Living together,Living together
mother_education,higher education,primary education (4th grade),primary education (4th grade),higher education,secondary education
father_education,higher education,primary education (4th grade),primary education (4th grade),5th to 9th grade,secondary education
mother_job,at_home,at_home,at_home,health,other
